Geometry optimizations in Python
================================

The *tblite* Python package allows to run extended tight binding (xTB) calculations directly in Python.
This tutorial demonstrates how to set up and run a geometry optimizations using GFN1-xTB.

Installing the package
----------------------

To start create a new Python environment using the mamba package manager.
We specify the packages we want to install in our environment file:

````yaml
name: xtb
channels:
- conda-forge
dependencies:
- ipykernel
- mdanalysis
- nglview
- pyberny
- tblite-python
````

Save the file as *environment.yml* and create the environment by running

````shell
mamba env create -n xtb -f environment.yml
mamba activate xtb
````

This will create a new environment called *xtb* and install all the necessary packages.
Make sure that *tblite* is available in your Python environment.
You can check this by opening a Python interpreter and importing the package

In [10]:
import tblite.interface as tb

tb.library.get_version()

(0, 4, 0)

Finding the right optimizer
---------------------------

The xTB calculator provided in *tblite* itself does not support geometry optimization by itself, but it can be used together with other geometry optimization packages.
One example would be the *pyberny* package which provides a general geometry optimization procedure.

In [2]:
%%writefile caffeine.xyz
24

C            1.07317        0.04885       -0.07573
N            2.51365        0.01256       -0.07580
C            3.35199        1.09592       -0.07533
N            4.61898        0.73028       -0.07549
C            4.57907       -0.63144       -0.07531
C            3.30131       -1.10256       -0.07524
C            2.98068       -2.48687       -0.07377
O            1.82530       -2.90038       -0.07577
N            4.11440       -3.30433       -0.06936
C            5.45174       -2.85618       -0.07235
O            6.38934       -3.65965       -0.07232
N            5.66240       -1.47682       -0.07487
C            7.00947       -0.93648       -0.07524
C            3.92063       -4.74093       -0.06158
H            0.73398        1.08786       -0.07503
H            0.71239       -0.45698        0.82335
H            0.71240       -0.45580       -0.97549
H            2.99301        2.11762       -0.07478
H            7.76531       -1.72634       -0.07591
H            7.14864       -0.32182        0.81969
H            7.14802       -0.32076       -0.96953
H            2.86501       -5.02316       -0.05833
H            4.40233       -5.15920        0.82837
H            4.40017       -5.16929       -0.94780

Overwriting caffeine.xyz


The optimizer provided by the *pyberny* package acts as an iterator to provide new geometry steps.
The geometry can be unpacked by iterating through it.
For xTB we need to separate the element symbols and the cartesian coordinates.

In [ ]:
import numpy as np
from berny import Berny, geomlib, angstrom

optimizer = Berny(geomlib.readfile("caffeine.xyz"))
geom = next(optimizer)
elements = [symbol for symbol, _ in geom]
coordinates = np.asarray([coordinate for _, coordinate in geom])

To visualize our starting geometry we use MDAnalysis and NGLView.
Loading the geometries into an MDAnalysis universe can be done by creating a new empty universe.
The atom types can be added as *name* topology attribute and the coordinates can be read by providing the array we just retrieved from the optimizer.

In [8]:
import MDAnalysis as mda
import nglview as nv

uni = mda.Universe.empty(len(elements), trajectory=True)
uni.add_TopologyAttr("name", elements)
uni.load_new(coordinates)

nv.show_mdanalysis(uni, gui=True)

NGLWidget()

To run the optimization we do need to compute the energy and forces, for this we will be using xTB via *tblite*.
We need to remember that coordinates in *tblite* might use a different unit than our optimizer, in this case *pyberny* uses Angstrom and *tblite* Bohr.
With the provided conversion factor we ensure that the coordinates are in the right unit.
While the energy unit Hartree is compatible for us, we need to account for the gradient unit, which is Hartree/Angstrom and convert the gradient accordingly.

In [ ]:
xtb = tb.Calculator("GFN2-xTB", tb.symbols_to_numbers(elements), coordinates * angstrom)
results = xtb.singlepoint()

optimizer.send((results.get("energy"), results.get("gradient") / angstrom))

------------------------------------------------------------
  cycle        total energy    energy error   density error
------------------------------------------------------------
      1     -41.75162462815  -4.2243950E+01   1.9479957E-01
      2     -42.11867876399  -3.6705414E-01   7.5972202E-02
      3     -42.14180557575  -2.3126812E-02   4.6343403E-02
      4     -42.14537345302  -3.5678773E-03   1.2550676E-02
      5     -42.14691416500  -1.5407120E-03   6.1305239E-03
      6     -42.14742063310  -5.0646810E-04   2.4358092E-03
      7     -42.14744770814  -2.7075046E-05   1.0726515E-03
      8     -42.14746243612  -1.4727977E-05   3.6117557E-04
      9     -42.14746301473  -5.7861259E-07   1.6698189E-04
     10     -42.14746312030  -1.0556378E-07   5.4777956E-05
     11     -42.14746315750  -3.7209666E-08   2.5307046E-05
     12     -42.14746315860  -1.0997780E-09   8.2333730E-06
------------------------------------------------------------

 total:                             

To run a full optimization we can use the optimizer in a loop retrieve the new coordinates.
From there we can update our xTB calculator with the new positions, evaluate the energy and gradient to pass them back to the optimizer.

In [ ]:
for geom in optimizer:
    coordinates = np.asarray([coordinate for _, coordinate in geom])
    xtb.update(positions=coordinates * angstrom)
    results = xtb.singlepoint(results)
    optimizer.send((results.get("energy"), results.get("gradient") / angstrom))

In this process we can record the energy, gradient as well as coordinates to store them for MDAnalysis to visualize the optimization progress and geometry change.